In [1]:
import sys
import rospy
import moveit_commander
import copy
import moveit_msgs.msg
import geometry_msgs.msg
from math import pi
from tf.transformations import *
from moveit_msgs.msg import RobotState, Constraints, JointConstraint, OrientationConstraint

In [2]:
rospy.init_node('bo_ur5', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'gripper']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])
planning_frame = move_group.get_planning_frame()

In [4]:
eef_link = move_group.get_end_effector_link()

In [5]:
robot_state = robot.get_current_state()
robot_state

joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, robotiq_85_left_knuckle_joint]
  position: [-0.0028408114901896653, 0.015576054779815607, 0.00010489376895428393, 0.005006323326917084, -0.000603027047008986, -0.018501657996647047, 0.0006713749782614187]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [6]:
def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [7]:
get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 265
    nsecs: 118000000
  frame_id: "world"
pose: 
  position: 
    x: 0.815700327705
    y: 0.196633529362
    z: 0.981761286106
  orientation: 
    x: -0.707896665185
    y: -0.706315171515
    z: 0.000766513357779
    w: 0.000776121724748


In [72]:
import geometry_msgs.msg
from geometry_msgs.msg import Quaternion, Point

orientation_0 = Quaternion(-0.707732860392,
                          -0.706466425662,
                           0.00311631615142,
                           0.00311067637957)

def bo_move_position( position, orientation=orientation_0 ):
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.orientation = orientation
    pose_goal.position = position
    move_group.set_pose_target(pose_goal)
    plan = move_group.go(wait=True)
    move_group.stop()
    move_group.clear_pose_targets()


In [ ]:
orientation_0 = Quaternion(-0.707732860392,
                          -0.706466425662,
                           0.00311631615142,
                           0.00311067637957)

Y_ENTRY_POINT = 0.45
Y_INSERT_LENGTH = 0.20


rack_0U = [ Point(-0.44, Y_ENTRY_POINT, 0.998+0.376),
           Point(-0.22, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.0, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.22, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.44, Y_ENTRY_POINT, 0.998+0.376)
          ]

rack_0L = [ Point(-0.44, Y_ENTRY_POINT, 0.998-0.376),
           Point(-0.22, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.0, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.22, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.44, Y_ENTRY_POINT, 0.998-0.376)
          ]

# go to the initial position 
bo_move_position(rack_0U[0])

for point in rack_0U:
    bo_move_position(point)
    bo_move_position(Point(point.x, point.y+Y_INSERT_LENGTH, point.z))
    bo_move_position(point)

for point in reversed(rack_0L):
    bo_move_position(point)
    bo_move_position(Point(point.x, point.y+Y_INSERT_LENGTH, point.z))
    bo_move_position(point)

# go to the initial position
bo_move_position(rack_0U[0])


동작이 가능한떄의 joint state 를 찾아서 해당 joint state 를 initialize 한 이후에 way point 들을 움직일 수 있도록 만들면 된다. 

In [9]:
constraints = Constraints()

constraints.name = "constraints1"

oc1 = OrientationConstraint()

# input orientation constraints
constraints.orientation_constraints.append(oc1)
move_group.set_path_constraints(constraints)


In [10]:
from geometry_msgs.msg import Quaternion

In [11]:
a = Quaternion(0,0,0,1)

In [12]:
a

x: 0
y: 0
z: 0
w: 1